In [ ]:
import pandas as pd
import numpy as np

#carga datasets
df_apre = pd.read_csv("aprehendidos_limpio_final.csv")
df_911 = pd.read_csv("ecu911_limpio_final.csv")

# Normalizacion de fechas
df_911["fecha"] = pd.to_datetime(df_911["fecha_dt"], errors="coerce").dt.date
df_apre["fecha"] = pd.to_datetime(df_apre["fecha"], errors="coerce").dt.date

#union de datasets por celdas 
df_union = df_apre.merge(
    df_911,
    on=["lat_grid", "lon_grid", "fecha"],
    how="outer",  # importante para no perder celdas sin datos
    suffixes=("_apre", "_911")
)

# ============================================================
# 3. RELLENAR VALORES FALTANTES DE TARGETS
# ============================================================

for col in ["conteo_delitos", "conteo_delitos_graves", "conteo_llamadas_riesgo"]:
    if col in df_union.columns:
        df_union[col].fillna(0, inplace=True)
    else:
        df_union[col] = 0  # si no existe, la creamos

#variables temporales

if "mes" not in df_union.columns:
    df_union["mes"] = pd.to_datetime(df_union["fecha"]).dt.month

if "dia" not in df_union.columns:
    df_union["dia"] = pd.to_datetime(df_union["fecha"]).dt.day

if "dia_semana" not in df_union.columns:
    df_union["dia_semana"] = pd.to_datetime(df_union["fecha"]).dt.dayofweek

# ELIMINAR FRANJA HORARIA PORQUE NO TIENE DATOS ÚTILES
if "franja_horaria" in df_union.columns:
    df_union.drop(columns=["franja_horaria"], inplace=True)
# features

features = [
    "lat_grid", "lon_grid",
    "mes", "dia", "dia_semana",
    "franja_horaria",
    "conteo_delitos",
    "conteo_delitos_graves",
    "conteo_llamadas_riesgo"
]

# asegurarse de que las columnas existan
features = [c for c in features if c in df_union.columns]

df_final = df_union[features].copy()

# guardar dataset final

df_final.to_csv("dataset_entrenamiento_final.csv", index=False)



print("Registros totales:", len(df_final))
print("Columnas:", df_final.columns.tolist())



C:\Users\fernando\AppData\Local\Temp\ipykernel_19292\793690625.py:8: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_apre = pd.read_csv("aprehendidos_limpio_final.csv")
C:\Users\fernando\AppData\Local\Temp\ipykernel_19292\793690625.py:32: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_union[col].fillna(0, inplace=True)



¡ARCHIVO FINAL DE ENTRENAMIENTO GENERADO CON ÉXITO!
Guardado como: dataset_entrenamiento_final.csv
Registros totales: 18605053
Columnas: ['lat_grid', 'lon_grid', 'mes', 'dia', 'dia_semana', 'conteo_delitos', 'conteo_delitos_graves', 'conteo_llamadas_riesgo']
